**Zackaria Mamdouh | Group 6 | Project 1**

_Written in collaboration with ChatGPT from OpenAI to improve understanding, assist with the explanation of the query, and enhance formatting and display of the queries._

**Complex Queries (from Pokemon Databases)**

**Query 1**

**Proposition**: The goal of this query is to compare the average Hit Points (HP), Attack, and Defense statistics for Pokémon across the first three generations.

**Tables**

The query involves three tables from a Pokémon database, one for each of the first three generations. Each table contains various statistics about Pokémon:

- `PokemonGen1.dbo.PokemonGen1`
- `PokemonGen2.dbo.PokemonGen2`
- `PokemonGen3.dbo.PokemonGen3`

**Columns**

The columns involved from each table are:

- `HP` (Hit Points)
- `Attack`
- `Defense`

**Predicate** 

For each generation's table, the query:

1. Calculates the average `HP`, `Attack`, and `Defense` values.
2. Labels the calculated averages with the respective generation name.

**Combined Statistics**

A Common Table Expression (CTE) named `CombinedStats` is used to unify the averages from all three tables. The final output of the query retrieves the generation and their corresponding average statistics for `HP`, `Attack`, and `Defense`.

The `UNION ALL` operator is used to combine the selections from the different generations into a single result set, which is then used to display the overall comparison.

In [ ]:
--QUERY 1-- This query combines and compares the average HP, Attack, and Defense stats across the three Pokémon generations.

WITH CombinedStats AS (
    -- Selects the average HP, Attack, and Defense stats from Generation 1
    SELECT 'Gen1' AS Generation, AVG(HP) AS AvgHP, AVG(Attack) AS AvgAttack, AVG(Defense) AS AvgDefense
    FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    -- Selects the average HP, Attack, and Defense stats from Generation 2
    SELECT 'Gen2', AVG(HP), AVG(Attack), AVG(Defense)
    FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    -- Selects the average HP, Attack, and Defense stats from Generation 3
    SELECT 'Gen3', AVG(HP), AVG(Attack), AVG(Defense)
    FROM PokemonGen3.dbo.PokemonGen3
)
-- The final SELECT fetches the combined average stats for each generation
SELECT Generation, AvgHP, AvgAttack, AvgDefense
FROM CombinedStats;

**Query 2**

**Proposition**
The aim of this query is to identify the most frequently occurring Pokémon type from the combined data of the first three generations.

**Tables**
This query involves three tables from a Pokémon database, each representing a different generation of Pokémon. The relevant tables and columns are:

- `PokemonGen1.dbo.PokemonGen1`
- `PokemonGen2.dbo.PokemonGen2`
- `PokemonGen3.dbo.PokemonGen3`

**Columns**

Each table includes two columns for Pokémon types:
- `Type1`
- `Type2`

**Predicate** 
The query performs the following actions:
1. Selects all non-null instances of `Type1` and `Type2` from the `PokemonGen1` table.
2. Repeats the selection for `Type1` and `Type2` from the `PokemonGen2` and `PokemonGen3` tables.
3. Uses the `UNION ALL` operator to combine these results into a single dataset.

**Determining the Most Common Type**
A subquery named `CombinedTypes` is created to hold the unified list of all types. The query then:
- Groups the results by `Type`.
- Counts the occurrences of each type.
- Orders the grouped count in descending order.

The `SELECT TOP 1` statement is used to retrieve only the most common type out of the dataset.

In [ ]:
-- --QUERY 2 --  This query finds the most common Pokémon type across all three generations.

SELECT TOP 1 Type, COUNT(*) AS TypeCount
FROM (
    -- Include all 'Type1' from Gen1
    SELECT Type1 AS Type FROM PokemonGen1.dbo.PokemonGen1 WHERE Type1 IS NOT NULL
    UNION ALL
    -- Include all 'Type2' from Gen1
    SELECT Type2 AS Type FROM PokemonGen1.dbo.PokemonGen1 WHERE Type2 IS NOT NULL
    UNION ALL
    -- Include all 'Type1' from Gen2
    SELECT Type1 AS Type FROM PokemonGen2.dbo.PokemonGen2 WHERE Type1 IS NOT NULL
    UNION ALL
    -- Include all 'Type2' from Gen2
    SELECT Type2 AS Type FROM PokemonGen2.dbo.PokemonGen2 WHERE Type2 IS NOT NULL
    UNION ALL
    -- Include all 'Type1' from Gen3
    SELECT Type1 AS Type FROM PokemonGen3.dbo.PokemonGen3 WHERE Type1 IS NOT NULL
    UNION ALL
    -- Include all 'Type2' from Gen3
    SELECT Type2 AS Type FROM PokemonGen3.dbo.PokemonGen3 WHERE Type2 IS NOT NULL
) AS CombinedTypes
GROUP BY Type
ORDER BY TypeCount DESC;

**Query 3**

**Proposition**:This query is designed to determine which Pokémon has the highest aggregate statistics, known as total stats (the sum of HP, Attack, Defense, SpecialAttack, SpecialDefense, and Speed), comparing across the first three generations of Pokémon.

**Tables** The query accesses three separate tables within a Pokémon database, with each table representing a different generation of Pokémon:

- `PokemonGen1.dbo.PokemonGen1`
- `PokemonGen2.dbo.PokemonGen2`
- `PokemonGen3.dbo.PokemonGen3`

**Columns**

From each table, it utilizes the following columns:

- `Name` (the name of the Pokémon)
- `HP` (Hit Points)
- `Attack`
- `Defense`
- `SpecialAttack`
- `SpecialDefense`
- `Speed`

**Predicate** The logic of the query executes these steps:

1. For each Pokémon in the `PokemonGen1` table, it computes the total of the aforementioned stats and assigns a label 'Gen1'.
2. It repeats the calculation for `PokemonGen2` and `PokemonGen3`, labeling the results 'Gen2' and 'Gen3', respectively.
3. It unifies the results from all three generations using `UNION ALL`.

**Aggregate and Grouping** The query then performs an aggregation to:

- Group the results by each Pokémon's name and generation, ensuring the max total is calculated within these groups.
- Use the `MAX` function to find the highest total stats for Pokémon within the same name and generation grouping.

In [ ]:
--QUERY 3 -- This query identifies the Pokémon with the highest total stats (sum of all individual stats) across all generations.

SELECT MAX(CombinedTotal.Total) AS MaxStat, CombinedTotal.Name, CombinedTotal.Generation
FROM (
    -- This subquery calculates the total stats for each Pokémon in Generation 1 and labels them 'Gen1'.
    SELECT Name, (HP + Attack + Defense + SpecialAttack + SpecialDefense + Speed) AS Total, 'Gen1' AS Generation FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    -- Similarly, it calculates the total stats for each Pokémon in Generation 2 and labels them 'Gen2'.
    SELECT Name, (HP + Attack + Defense + SpecialAttack + SpecialDefense + Speed), 'Gen2' FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    -- And does the same for Generation 3, labeling them 'Gen3'.
    SELECT Name, (HP + Attack + Defense + SpecialAttack + SpecialDefense + Speed), 'Gen3' FROM PokemonGen3.dbo.PokemonGen3
) AS CombinedTotal
-- Grouping the results by Pokémon name and generation.
GROUP BY CombinedTotal.Name, CombinedTotal.Generation;

**Query 4** 

**Proposition**: This query is structured to analyze the average Speed of Pokémon, distinguishing between those with a single type (Single) and those with two types (Dual), across three generations.

**Tables**
The data is sourced from three different tables in the Pokémon database, each corresponding to a distinct generation of Pokémon:

- `PokemonGen1.dbo.PokemonGen1`
- `PokemonGen2.dbo.PokemonGen2`
- `PokemonGen3.dbo.PokemonGen3`

**Columns**

The columns assessed from these tables are:
- `Speed` (a statistic of the Pokémon)
- `Type2` (the second type of the Pokémon, if present)

**Predicate**
The query executes the following operations:
1. Constructs a case classification within a subquery for each Pokémon in the `PokemonGen1` table to determine if the Pokémon is Single or Dual-typed based on the presence or absence of `Type2`.
2. It replicates this classification for `PokemonGen2` and `PokemonGen3`.
3. All three selections are unified using `UNION ALL` to include all generations in a common format.

**Calculating Average Speed**
Post-unification, the query:
- Groups the data by `Generation` and `TypeCategory`.
- Calculates the average `Speed` for each group using the `AVG` function.

In [ ]:
-- QUERY 4 --  This query calculates the average speed of Pokémon based on their type classification (Single or Dual) for each generation.

SELECT Generation, TypeCategory, AVG(Speed) AS AverageSpeed
FROM (
    -- Create a subquery that selects generation, type category, and speed for Gen1 Pokémon
    SELECT 'Gen1' AS Generation, 
           CASE WHEN Type2 IS NOT NULL THEN 'Dual' ELSE 'Single' END AS TypeCategory, 
           Speed 
    FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    -- Repeat the same selection for Gen2 Pokémon, classifying type category based on the presence of Type2
    SELECT 'Gen2', 
           CASE WHEN Type2 IS NOT NULL THEN 'Dual' ELSE 'Single' END, 
           Speed 
    FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    -- Repeat the selection for Gen3 Pokémon, continuing the type category classification
    SELECT 'Gen3', 
           CASE WHEN Type2 IS NOT NULL THEN 'Dual' ELSE 'Single' END, 
           Speed 
    FROM PokemonGen3.dbo.PokemonGen3
) AS CombinedTypes
-- Group the results by generation and type category
GROUP BY Generation, TypeCategory;

**Query 5**

**Proposition**: This query aims to count the number of legendary Pokémon in each generation, identifying them by the presence of the 'Pressure' ability, which is considered characteristic of legendary Pokémon.

**Tables**
The query accesses data from three tables within a Pokémon database, each corresponding to a different generation:

- `PokemonGen1.dbo.PokemonGen1`
- `PokemonGen2.dbo.PokemonGen2`
- `PokemonGen3.dbo.PokemonGen3`

**Columns**

Columns involved in the query are:
- `Name` (the name of the Pokémon)
- `Ability1`, `Ability2`, `Ability3` (different abilities a Pokémon may have)

**Predicate** 
The query consists of the following steps:
1. Selects Pokémon from the `PokemonGen1` table that have the 'Pressure' ability in any of their ability slots.
2. Applies the same selection criteria to the `PokemonGen2` and `PokemonGen3` tables.
3. Combines the results from all three generations using `UNION ALL`, forming a unified dataset of legendary Pokémon across generations.

**Counting Legendary Pokémon**
In the final stage, the query:
- Groups the collected data by `Generation`.
- Uses the `COUNT` function to determine the total number of legendary Pokémon for each generation.

**Purpose**
The outcome of this query provides insight into the distribution of legendary Pokémon, denoted by the 'Pressure' ability, across the first three generations of the Pokémon series.

In [ ]:
-- QUERY 5 --  This query calculates the number of legendary Pokémon in each generation by searching for the 'Pressure ability that is exclusive to legendary Pokemon.
SELECT Generation, COUNT(*) AS LegendaryCount
FROM (
    -- Selects all Pokémon from Generation 1 that have the 'Pressure' ability, which is assumed to be a marker of legendary Pokémon.
    SELECT 'Gen1' AS Generation, Name 
    FROM PokemonGen1.dbo.PokemonGen1 
    WHERE Ability1 = 'Pressure' OR Ability2 = 'Pressure' OR Ability3 = 'Pressure'
    UNION ALL
    -- Selects all Pokémon from Generation 2 with the 'Pressure' ability.
    SELECT 'Gen2', Name 
    FROM PokemonGen2.dbo.PokemonGen2 
    WHERE Ability1 = 'Pressure' OR Ability2 = 'Pressure' OR Ability3 = 'Pressure'
    UNION ALL
    -- Selects all Pokémon from Generation 3 with the 'Pressure' ability.
    SELECT 'Gen3', Name 
    FROM PokemonGen3.dbo.PokemonGen3
    WHERE Ability1 = 'Pressure' OR Ability2 = 'Pressure' OR Ability3 = 'Pressure'
) AS LegendaryPokemons -- The subquery results are aliased as LegendaryPokemons.
GROUP BY Generation -- Groups the results by generation to prepare for the count.
-- The count function is used to count the number of legendary Pokémon in each group.

**Query 6**

**Proposition**: This query aims to list Pokémon from all generations, ordered by their Defense to Attack ratio in descending order, to understand how defensive each Pokémon is relative to its offensive capabilities.

**Tables**
The query utilizes data from three tables within a Pokémon database, corresponding to different Pokémon generations:

- `PokemonGen1.dbo.PokemonGen1`
- `PokemonGen2.dbo.PokemonGen2`
- `PokemonGen3.dbo.PokemonGen3`

**Columns**

The columns used from these tables are:
- `Name` (the Pokémon's name)
- `Defense` (the Pokémon's defense statistic)
- `Attack` (the Pokémon's attack statistic)

**Predicate** 
The query executes the following steps:
1. In a subquery, it selects the `Name`, `Defense`, and `Attack` stats from the `PokemonGen1` table and labels the generation as 'Gen1'.
2. It repeats this selection for `PokemonGen2` and `PokemonGen3`, labeling the results 'Gen2' and 'Gen3', respectively.
3. The results from all three generations are combined into a single dataset using `UNION ALL`.

**Calculating Defense to Attack Ratio**
In the main query:
- It creates a new column `DefenseAttackRatio` by dividing the `Defense` stat by the `Attack` stat for each Pokémon. The division is cast to FLOAT to ensure an accurate decimal ratio.
- The final results are then ordered by this ratio in descending order (`ORDER BY DefenseAttackRatio DESC`), so Pokémon with higher defensive capabilities relative to their attack are listed first.

In [ ]:
--QUERY 6 --  This query lists the Pokémon from all generations ordered by their Defense to Attack ratio in descending order.

-- Create a virtual table that combines the Name, Defense, and Attack stats for all Pokémon across generations.
SELECT Generation, Name, Defense, Attack, CAST(Defense AS FLOAT) / Attack AS DefenseAttackRatio
FROM (
    -- Select Name, Defense, and Attack stats from Generation 1 Pokémon.
    SELECT 'Gen1' AS Generation, Name, Defense, Attack FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    -- Repeat the selection for Generation 2.
    SELECT 'Gen2', Name, Defense, Attack FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    -- Repeat the selection for Generation 3.
    SELECT 'Gen3', Name, Defense, Attack FROM PokemonGen3.dbo.PokemonGen3
) AS CombinedStats
-- Order the results by the Defense to Attack ratio, from highest to lowest.
ORDER BY DefenseAttackRatio DESC;

**Query 7**

**Proposition**: This query aims to list Pokémon from all generations, ordered by their Defense to Attack ratio in descending order, to understand how defensive each Pokémon is relative to its offensive capabilities.

**Tables**
The query utilizes data from three tables within a Pokémon database, corresponding to different Pokémon generations:
- `PokemonGen1.dbo.PokemonGen1`
- `PokemonGen2.dbo.PokemonGen2`
- `PokemonGen3.dbo.PokemonGen3`

**Columns**

The columns used from these tables are:
- `Name` (the Pokémon's name)
- `Defense` (the Pokémon's defense statistic)
- `Attack` (the Pokémon's attack statistic)

**Predicate** 
The query executes the following steps:
1. In a subquery, it selects the `Name`, `Defense`, and `Attack` stats from the `PokemonGen1` table and labels the generation as 'Gen1'.
2. It repeats this selection for `PokemonGen2` and `PokemonGen3`, labeling the results 'Gen2' and 'Gen3', respectively.
3. The results from all three generations are combined into a single dataset using `UNION ALL`.

**Calculating Defense to Attack Ratio**
In the main query:
- It creates a new column `DefenseAttackRatio` by dividing the `Defense` stat by the `Attack` stat for each Pokémon. The division is cast to `FLOAT` to ensure an accurate decimal ratio.
- The final results are then ordered by this ratio in descending order (`ORDER BY DefenseAttackRatio DESC`), so Pokémon with higher defensive capabilities relative to their attack are listed first.


In [ ]:
--QUERY 7 -- Query to find the top 5 Pokémon with the highest average combined Defense and Special Defense across all generations.

SELECT TOP 5 Name, MainType, AVG(Defense + SpecialDefense) as AverageDefense
FROM (
    -- Combining Pokémon data from Generation 1. Selecting Name, primary Type, Defense, and SpecialDefense.
    SELECT Name, Type1 as MainType, Defense, SpecialDefense FROM PokemonGen1.dbo.PokemonGen1
    UNION ALL
    -- Repeating the process for Generation 2 Pokémon.
    SELECT Name, Type1, Defense, SpecialDefense FROM PokemonGen2.dbo.PokemonGen2
    UNION ALL
    -- Repeating the process for Generation 3 Pokémon.
    SELECT Name, Type1, Defense, SpecialDefense FROM PokemonGen3.dbo.PokemonGen3
) AS CombinedPokemons
-- Grouping the results by Pokémon Name and their primary Type.
GROUP BY Name, MainType
-- Ordering the Pokémon based on their average combined Defense and SpecialDefense in descending order.
ORDER BY AverageDefense DESC;


**Below this line is the 13 medium queries using Northwinds and AdventureWorks**

**\-------------------------------------------------------------------------------------------------**